<H1>Neighbourhoods in Toronto</H1>

<h2>Part I</h2>

Lets import required packages

In [125]:
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim

Read html page into panda

In [126]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = pd.read_html(url)

In [127]:
df = page[0]
df.shape

(180, 3)

In [128]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove Not assigned Borough rows

In [129]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df.shape

(103, 3)

Check any duplicate Postal Codes

In [130]:
len(df['Postal Code'].unique()) != len(df['Postal Code'])

False

There are no any duplicate Postal Code rows

In [131]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check any not assigned Neighborhoods in data frame

In [132]:
df["Neighborhood"].isna().sum()

0

There are no Neighborhoods which are not assigned or NaN

In [133]:
#Check Shape of Dataframe
df.shape

(103, 3)

#========================================================#

<h2>Part II</h2>

Lets define a mothod to find Geo Cordinates

In [137]:
def get_geocodination(postal_code):
    
    lat_lng_coords = None
    
    g = geocoder.arcgis('{}, Toronto, Canada'.format(postal_code))
    lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
            
    return latitude,longitude

Get Latitude and Logitude by calling above funtion and add them to dataframe

In [139]:
df['Latitude'],df['Longitude'] = zip(*df['Postal Code'].apply(get_geocodination))

In [142]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [143]:
#Checking any Null values in dataframe
df.isna().sum()

Postal Code     0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [144]:
df.shape

(103, 5)

Thats the end of part II

#========================================================#